In [1]:
import sys
import os
import numpy as np

# path to folder to CityGeoTools
folder = "/home/gk/code/CITY_GEO_TOOLS_DOCKER/CityGeoTools"
sys.path.append(folder)

from metrics.data import CityInformationModel as BaseModel

/home/gk/code/CITY_GEO_TOOLS_DOCKER/.venv/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
local_crs= 32645

### Create CityInformationModel

To define own City Information Model, you need to create an instance of CityInformationModel class with specified attributes: 
   
**city_name** - string (name of city which you are going to work with)  
**city_crs** - integer (EPSG projection of city)   
**cwd** - relative path to CityGeoTools library 


In [3]:
city_model = BaseModel.CityInformationModel(city_name="Томск", city_crs=local_crs, cwd="../")
city_model

The object of class CityInformationModel contains attributes describing data required for successful method call.  
After creating own model these attributes equal to **None** by default.

In [4]:
city_model.get_all_attributes()

{'city_name': 'Томск',
 'city_crs': 32645,
 'city_id': typing.Optional[int],
 'cwd': '../',
 'mode': 'user_mode',
 'MobilityGraph': None,
 'Buildings': None,
 'Services': None,
 'PublicTransportStops': None,
 'ServiceTypes': None,
 'RecreationalAreas': None,
 'Blocks': None,
 'Municipalities': None,
 'AdministrativeUnits': None,
 'ValueTypes': None,
 'SocialGroups': None,
 'SocialGroupsValueTypesLivingSituations': None,
 'LivingSituationsCityServiceTypes': None,
 'LivingSituations': None,
 'methods': <metrics.data.DataValidation.DataValidation at 0x7efda85b63a0>}

### Update layers

To use CityGeoTools methods, it is necessary to load prepaid data to the model specifying attribute names and paths to files.  
Only **json**, **geojson** and **graphml** file formats are available for loading. Before data loading, all methods are marked as unavailable.  
Exception is weighted voronoi method, since it doesn't require any urban data.

In [5]:
print("All methods implemented in CityGeoTools:\n")

all_methods = city_model.methods.get_list_of_methods()
available_methods = city_model.methods.get_list_of_available_methods()

for method in all_methods:
    if method in available_methods:
        print(method, "--> available")
    else:
        print(method, "--> unavailable")


All methods implemented in CityGeoTools:

traffics_calculation --> unavailable
visibility_analysis --> unavailable
weighted_voronoi --> available
blocks_clusterization --> unavailable
services_clusterization --> unavailable
spacematrix --> unavailable
mobility_analysis --> unavailable
diversity --> unavailable
collocation_matrix --> unavailable
coverage_zones --> unavailable
master_plan --> unavailable
blocks_accessibility --> unavailable


In [6]:
# pip install pyarrow

In [7]:
city_name='Томск'
import geopandas as gpd

blocks = gpd.read_parquet(f'{city_name}_blocks.parquet')
blocks = blocks.drop(columns=['landuse'])
blocks['is_polygon'] = blocks['geometry'].apply(lambda x : type(x).__name__ == 'Polygon')
blocks = blocks.loc[blocks['is_polygon']].reset_index(drop=True)

In [8]:
buildings = gpd.read_parquet('migr_proj_buildings_cities.parquet').to_crs(local_crs)
mask = buildings['city'] == city_name
buildings = buildings.loc[mask]
buildings.rename(columns={'building_id': 'id'}, inplace=True)
buildings.to_file('buildings.geojson')

In [9]:
services = gpd.read_file('services_ALL.geojson')
mask = services['city'] == city_name
services = services.loc[mask]
services.to_crs(local_crs, inplace=True)
services.rename(columns={'building_id': 'id', 'city_service_type_code': 'service_code'}, inplace=True)
services.to_file('services.geojson')

In [10]:
services

,functional_object_id,id,city_service_type_id,city_id,city,city_service_type,service_code,city_function_code,capacity,is_capacity_real,geometry
9,1427731,1883562,2,31,Томск,Школа,schools,education,601,False,POINT (377203.908 6260962.511)
11,1424184,1898375,1,31,Томск,Детский сад,kindergartens,education,94,False,POINT (369670.525 6260657.553)
19,1422830,1858999,50,31,Томск,Клиника,clinics,health,589,False,POINT (374724.553 6259909.592)
22,1424326,1895457,1,31,Томск,Детский сад,kindergartens,education,144,False,POINT (376667.432 6261608.754)
29,1427728,1881243,2,31,Томск,Школа,schools,education,608,False,POINT (378178.274 6264934.068)
...,...,...,...,...,...,...,...,...,...,...,...
1029,1424196,1886573,1,31,Томск,Детский сад,kindergartens,education,68,False,POINT (375216.538 6261124.813)
1038,1427747,1845112,2,31,Томск,Школа,schools,education,847,False,POINT (374748.112 6259501.654)
1039,1424013,1879411,6,31,Томск,Больница,hospitals,health,5366,False,POINT (373867.857 6263783.780)
1040,1424197,1860063,1,31,Томск,Детский сад,kindergartens,education,143,False,POINT (375754.481 6265073.968)


In [11]:
blocks = gpd.read_parquet(f'{city_name}_blocks.parquet')
blocks.to_file(f'{city_name}_blocks.geojson')

In [12]:
# GeoJSON
city_model.update_layer("Buildings", "buildings.geojson")
city_model.update_layer("Services", "services.geojson")
city_model.update_layer("Blocks", f"{city_name}_blocks.geojson")



#GraphML
city_model.update_layer("MobilityGraph", f"{city_name}.graphml")

Validation of Buildings layer...
Buildings layer loaded successfully!
Validation of Services layer...
Services layer loaded successfully!
Validation of Blocks layer...
Blocks layer loaded successfully!
Validation of MobilityGraph layer...
MobilityGraph layer loaded successfully!


In [13]:
#JSON
# city_model.update_layer("ServiceTypes", "service_types.json") 

In [14]:
print("All methods implemented in CityGeoTools:\n")

all_methods = city_model.methods.get_list_of_methods()
available_methods = city_model.methods.get_list_of_available_methods()

for method in all_methods:
    if method in available_methods:
        print(method, "--> available")
    else:
        print(method, "--> unavailable")

All methods implemented in CityGeoTools:

traffics_calculation --> unavailable
visibility_analysis --> unavailable
weighted_voronoi --> available
blocks_clusterization --> unavailable
services_clusterization --> unavailable
spacematrix --> unavailable
mobility_analysis --> available
diversity --> unavailable
collocation_matrix --> unavailable
coverage_zones --> unavailable
master_plan --> unavailable
blocks_accessibility --> unavailable


In [15]:
import psycopg2 as pg
city_model.engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")

In [16]:
service_types_d = services.rename(columns={'city_service_type_code': 'code'})

In [17]:
import pandas as pd
q = "select id, code, name, public_transport_time_normative, walking_radius_normative "\
    "from city_service_types "\
    "where code in ('kindergartens', 'schools', 'clinics', 'hospitals', 'multifunctional_centers') "\
    ""
gdf  = pd.read_sql(q, con=city_model.engine)
gdf

/tmp/ipykernel_417616/2949202789.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  gdf  = pd.read_sql(q, con=city_model.engine)


,id,code,name,public_transport_time_normative,walking_radius_normative
0,2,schools,Школа,NaN,500.0
1,6,hospitals,Больница,60.0,NaN
2,1,kindergartens,Детский сад,NaN,300.0
3,50,clinics,Клиника,NaN,500.0
4,70,multifunctional_centers,МФЦ,20.0,NaN


In [18]:
city_model.ServiceTypes = gdf

In [19]:
city_model.ServiceTypes

,id,code,name,public_transport_time_normative,walking_radius_normative
0,2,schools,Школа,NaN,500.0
1,6,hospitals,Больница,60.0,NaN
2,1,kindergartens,Детский сад,NaN,300.0
3,50,clinics,Клиника,NaN,500.0
4,70,multifunctional_centers,МФЦ,20.0,NaN


In [26]:
city_model.Buildings

,geometry,id,functional_object_id,is_living,city,building_area,living_area,storeys_count,population_balanced,city_id
0,POINT (inf inf),1838273,None,0,Томск,726.1850,NaN,1.0,0,31
1,POINT (inf inf),1838275,None,1,Томск,174.2170,243.9040,2.0,8,31
2,POINT (inf inf),1838277,None,0,Томск,289.4060,NaN,1.0,0,31
3,POINT (inf inf),1838279,None,0,Томск,731.0240,NaN,1.0,0,31
4,POINT (inf inf),1838281,None,0,Томск,117.8280,NaN,1.0,0,31
...,...,...,...,...,...,...,...,...,...,...
38463,POINT (inf inf),1919368,None,1,Томск,55.9284,39.1499,1.0,0,31
38464,POINT (inf inf),1919369,None,1,Томск,62.2711,43.5898,1.0,1,31
38465,POINT (inf inf),1919370,None,1,Томск,38.7184,27.1029,1.0,1,31
38466,POINT (inf inf),1919371,None,1,Томск,61.3945,42.9761,1.0,4,31


In [31]:
Provisions_class.Provisions["kindergartens"]['buildings']

In [39]:
from metrics.calculations.city_provision import CityProvision
Provisions_class = CityProvision(city_model,
                             service_types = ["kindergartens"],
                             valuation_type = "normative",
                             year = 2023, 
                             user_changes_services = None,
                             user_selection_zone = None)

Provisions_class.Provisions["buildings"] = city_model.Buildings
Provisions_class.Provisions["services"] = city_model.Services

r = Provisions_class._calculate_provisions(Provisions_class.Provisions, service_type="kindergartens", calculation_type="gravity")                             

/home/gk/code/CITY_GEO_TOOLS_DOCKER/CityGeoTools/metrics/calculations/city_provision.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  self.demands = pd.read_sql(f'''SELECT building_id as id, {", ".join(f"{service_type}_service_demand_value_{self.valuation_type}" for service_type in service_types)}


### Deal with ValidationError

It should be noted that all loaded data MUST match specification for certain method to successfully call it.  
Otherwise, you will encounter the ValidationError even though the file is saved into attribute.   
To know why loaded file does not match specification, see attribute **message** that is available trought city_model.methods  

In [10]:
from metrics.calculations import spacematrix
spacematrix.Spacematrix(city_model).get_morphotypes()

ValidationError: Layers Blocks do not match specification.

In [11]:
city_model.methods.spacematrix.message

{'Buildings': 'Layer matches specification',
 'Blocks': "79.0 is not of type 'integer'"}